# Week8：作業回饋

## 作業目的
1. 讓同學熟悉函式定義（`def funcX`）、函式呼叫（`funcX()`）的語法，以及多個函式如何在一個程式中分工合作。
2. 再次讓同學練習如何使用迴圈，以及嘗試閱讀說明文件（例如`datetime.strptime`）。
3. 觀察資料結構如何幫助程式操作資料，例如這個作業的`ltime_dict`。

## 參考作法

為讓程式碼長度縮短，以下僅呈現作業需實作的`group_last_time`函式，以及相對應的呼叫程式

In [1]:
from datetime import datetime

## Data Section
## =============

events = [
    ("2018/10/25", "寫日記", 30),
    ("2018/10/23", "7-11", 1),
    ("2018/9/27", "寫日記", 30),
    ("2018/9/28", "寫日記", 20),
    ("2018/9/29", "寫日記", 40),
    ("2018/9/29", "總圖", 3),
    ("2018/9/30", "寫日記", 25),
    ("2018/10/21", "7-11", 2),
    ("2018/10/22", "總圖", 3),
    ("2018/10/23", "總圖", 2)
]

event_info = {
    "寫日記": {
        "detail": "寫日記花的時間",
        "unit": "分鐘"},
    "7-11": {
        "detail": "在7-11蒐集到的點數",
        "unit": "點"},
    "總圖": {
        "detail": "在總圖唸書的時間",
        "unit": "小時"}
}

## -----------------------
##   Function Definition
## -----------------------
def group_last_time(ev_data, ev_info):
    ltime_dict = {}
    for ev_x in ev_data:
        ev_category = ev_x[1]
        ltime_day = ltime_dict.get(ev_category, 36500)
        date_x = datetime.strptime(ev_x[0], "%Y/%m/%d")
        date_diff = datetime.now() - date_x
        ltime_day = min(ltime_day, date_diff.days)
        ltime_dict[ev_category] = ltime_day

    output_text = format_group_message(
        "[{category}]，最近一次是{value}天前\n", ltime_dict, ev_info)
    return output_text

def format_group_message(template, group_dict, ev_info):
    output_text = ""
    for group_cat, group_val in group_dict.items():
        group_detail = ev_info[group_cat]["detail"]
        group_unit = ev_info[group_cat]["unit"]
        output_text += template.format(
            category=group_cat,
            detail=group_detail,
            value=group_val,
            unit=group_unit
        )
    return output_text

## ---------------------
##  Main Entry Function
## ---------------------

def main():
    outStr = group_last_time(events, event_info)
    print(outStr)
    
## Calling main() function
main()


[寫日記]，最近一次是12天前
[7-11]，最近一次是14天前
[總圖]，最近一次是14天前



## 相關問題

### ltime_dict是做什麼的？

`ltime_dict`是一個暫存變項，負責收集「迭代到目前為止的成果」用的。以下的程式和作業相似，但加了一些debug print，希望能幫助同學理解它在程式中的作用。

以下這個例子只有2個事件類別（寫日記、7-11），共4個事件。在走迴圈時，第一次迭代（iteration）遇到的是「寫日記」，距離現在（今天是2018/11/6）5天前，所以在`ltime_dict`中會記錄一筆`{'寫日記': 5}`。接著第二次迭代中，遇到「7-11」，距離現在1天前，所以`ltime_dict`會再記錄一筆鍵值對，變成`{'寫日記': 5, '7-11': 1}`。第三次迭代，遇到是「寫日記」，距離現在3天前。這時候這個天數比`ltime_dict`裡的「寫日記」對應到的數值`5`小，所以就取代掉它。這時`ltime_dict`裡的值是`{'寫日記': 3, '7-11': 1}`。最後一次迭代，遇到「7-11」，距離現在2天前，這個值沒有比`ltime_dict`裡7-11所對應到的數值`1`小，所以還是保留一樣的值`{'寫日記': 3, '7-11': 1}`。於是最後的輸出值就是`{'寫日記': 3, '7-11': 1}`。

`ltime_dict`扮演的角色很像W7的`line_buffer`，只是又更「活」了一點。`ltime_dict`會動態地（dynamically）「看」到不同的活動類別，而新增不同的key；同時，如果遇到新的事件，其距離現在的天數比`ltime_dict`裡該事件所對應的暫存天數小，它就「知道」要把它換過來。於是最後`ltime_dict`的內容就是分群（分各個活動類別），以及彙總（每個類別的最小值）的結果。

In [2]:
events_dbg = [
    ("2018/11/1", "寫日記", 30),
    ("2018/11/5", "7-11", 1),
    ("2018/11/3", "寫日記", 25),
    ("2018/11/4", "7-11", 2)
]

def group_last_time(ev_data, ev_info):
    ltime_dict = {}
    for ev_x in ev_data:
        ev_category = ev_x[1]
        ltime_day = ltime_dict.get(ev_category, 36500)
        date_x = datetime.strptime(ev_x[0], "%Y/%m/%d")
        date_diff = datetime.now() - date_x
        ltime_day = min(ltime_day, date_diff.days)
        ltime_dict[ev_category] = ltime_day
        print("[DEBUG] ev_x: {}".format(ev_x))
        print("[DEBUG] ltime_dict: {}".format(ltime_dict))

    return ltime_dict

def main():
    print("Calling group_last_time(): {}".format(group_last_time(events_dbg, event_info)))

## Calling main() function
main()

[DEBUG] ev_x: ('2018/11/1', '寫日記', 30)
[DEBUG] ltime_dict: {'寫日記': 5}
[DEBUG] ev_x: ('2018/11/5', '7-11', 1)
[DEBUG] ltime_dict: {'寫日記': 5, '7-11': 1}
[DEBUG] ev_x: ('2018/11/3', '寫日記', 25)
[DEBUG] ltime_dict: {'寫日記': 3, '7-11': 1}
[DEBUG] ev_x: ('2018/11/4', '7-11', 2)
[DEBUG] ltime_dict: {'寫日記': 3, '7-11': 1}
Calling group_last_time(): {'寫日記': 3, '7-11': 1}


### datetime.strptime是什麼？

作業裡每個events的日期資料（如"2018/10/23"）是一個字串，而datetime是另外一個「代表時間和日期的物件」，所以從字串到datetime物件，需要用[datetime.strptime][strptime_doc]方法。strptime和strftime兩個方法中，p代表parse，也就是從字串剖析（parse）成時間；另一個f代表format，也就是從時間格式化（format）成字串。在這裡，我們要從代表日期的字串變成datetime物件，所以這裡需要用到的是strptime方法。strptime跟strftime一樣都需要給日期格式字串。我們這裡用的日期資料是「四位數的西元年」（%Y）、月份數字（%m）、日期數字（%d）。這就是作業提示的日期格式字串（"%Y/%m/%d"）中的三個符號。這三個符號中間用`/`分隔，是因為events的日期資料是用斜線（`/`）分隔。其他可能常見的日期字串以及他們的日期格式字串如下：

[strptime_doc]:https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime

In [3]:
datetime.strptime("20180314", "%Y%m%d")

datetime.datetime(2018, 3, 14, 0, 0)

In [4]:
datetime.strptime("180314", "%y%m%d")

datetime.datetime(2018, 3, 14, 0, 0)

In [5]:
datetime.strptime("2018-03-14", "%Y-%m-%d")

datetime.datetime(2018, 3, 14, 0, 0)

### 附註：電腦裡的時間
就如人類的文字對電腦而言很陌生，人類世界的時間對電腦而言也很陌生。人們平常表達日期時間的方式並不固定，常間的格式就有年月日、日月年、月日年等表達方法，甚至還有可能有加星期幾，沒有加星期幾等等不同的表達方式。為了讓程式能方便操作、剖析、格式化這些日期時間概念，Python在`datetime`模組（module）中有許多相關的「類別」（class）。這些類別包括描述日期的`date`、描述時間的`time`、描述日期和時間的`datetime`、描述時間差異的`timedelta`，甚至還有描述時區的`tzinfo`和`timezone`。這些模組、類別看似複雜，但它們只是Python為了「適應」複雜的人類世界所提供的方便工具。

## 延伸挑戰

這週的延伸挑戰真的「很有挑戰性」。以下提供兩種想法參考。就如同上一週的作業回饋所提的，這堂課不會區分實作方法的好壞。同學能夠用程式講出一個解決問題的「故事」是最重要的。

### 用暫存變項檢查並記錄連續

In [6]:
def group_longest(ev_data, ev_info):
    buf_dict = {}
    def get_def_buf():
        return {"longest": 1, "lastdate": None, "continued": 0}
    ev_sorted = [(datetime.strptime(x[0], "%Y/%m/%d"),
                  x[1], x[2]) for x in ev_data]
    ev_sorted.sort(key=lambda x: (x[1], x[0]))

    for ev_x in ev_sorted:
        ev_category = ev_x[1]
        date_x = ev_x[0]
        ev_buf = buf_dict.setdefault(ev_category, get_def_buf())

        ev_last = ev_buf["lastdate"]
        if ev_last and (date_x - ev_last).days == 1:
            ev_buf["continued"] = ev_buf["continued"] + 1
            ev_buf["longest"] = max(ev_buf["longest"], ev_buf["continued"])
        else:
            ev_buf["continued"] = 1
        ev_buf["lastdate"] = date_x
        buf_dict[ev_category] = ev_buf

    cont_dict = {k: ev_buf["longest"] for k, ev_buf in buf_dict.items()}
    output_text = format_group_message(
        "[{category}]，最長連續紀錄是{value}天\n", cont_dict, ev_info)
    return output_text
print(group_longest(events, event_info))

[7-11]，最長連續紀錄是1天
[寫日記]，最長連續紀錄是4天
[總圖]，最長連續紀錄是2天



### 用輔助變項定義連續

In [7]:
from itertools import groupby
from datetime import timedelta

def group_longest_iter(ev_data, ev_info):
    ev_sorted = [(datetime.strptime(x[0], "%Y/%m/%d"),
                  x[1], x[2]) for x in ev_data]
    ev_sorted.sort(key=lambda x: (x[1], x[0]))

    cont_dict = {}
    for grp, grp_iter in groupby(ev_sorted, key=lambda x: x[1]):
        grp_list = list(grp_iter)
        shifted = [grp_list[-1]] + grp_list[:-1]
        cumsum = 0
        cumsum_list = []
        for x0, x1 in zip(grp_list, shifted):
            diff = x0[0] - x1[0]
            cumsum += int(diff != timedelta(days=1))
            cumsum_list.append(cumsum)

        seq_groups = groupby(cumsum_list)
        seq_lens = [len(list(seq_iter)) for _, seq_iter in seq_groups]
        cont_dict[grp] = max(seq_lens)

    output_text = format_group_message(
        "[{category}]，最長連續紀錄是{value}天\n", cont_dict, ev_info)

    return output_text
print(group_longest_iter(events, event_info))

[7-11]，最長連續紀錄是1天
[寫日記]，最長連續紀錄是4天
[總圖]，最長連續紀錄是2天

